In [1]:
pip install scx>=1.2.1 #Note: This may throw an Error in Google Colab since Colab uses an outdated duckdb package. This file should still work in Colab without issues.

Note: you may need to restart the kernel to use updated packages.


<p class="MsoNormal">Food-On-Wheels is a food delivery company operating in Alabama. They sell standard boxes containing eight full healthy meals. Boxes are prepared in two kitchens and delivered by bike to five city areas. <o:p></o:p></p>
<p><b>Table 1: Demand in boxes</b></p>
<table width="617" height="203" style="width: 30%;">
<tbody>
<tr>
<td width="10%" style="text-align: center; border: 1px solid black;"></td>
<td width="30%" style="text-align: center; border: 1px solid black;">Demand (in boxes per day)</td>
</tr>
<tr>
<td width="10%" style="text-align: center; border: 1px solid black;">Area 1</td>
<td width="30%" style="text-align: center; border: 1px solid black;">101</td>
</tr>
<tr>
<td width="10%" style="text-align: center; border: 1px solid black;">Area 2</td>
<td width="30%" style="text-align: center; border: 1px solid black;">61</td>
</tr>
<tr>
<td width="10%" style="text-align: center; border: 1px solid black;">Area 3</td>
<td width="30%" style="text-align: center; border: 1px solid black;">101</td>
</tr>
<tr>
<td width="10%" style="text-align: center; border: 1px solid black;">Area 4</td>
<td width="30%" style="text-align: center; border: 1px solid black;">87</td>
</tr>
<tr>
<td width="10%" style="text-align: center; border: 1px solid black;">Area 5</td>
<td width="30%" style="text-align: center; border: 1px solid black;">50</td>
</tr>
</tbody>
</table>
<p class="MsoNormal"></p>
<p class="MsoNormal">Each kitchen has a limited daily capacity to prepare boxes. Kitchen 1 can prepare 250 boxes a day, and Kitchen 2 can prepare 350 boxes per day.<o:p></o:p></p>
<p></p>
<p class="MsoNormal">Boxes are bulky, therefore a bike can only deliver one box at a time. There are many bikers in the city that work for Food-On-Wheels on call, so availability of bikes to do the deliveries can be considered unlimited. <o:p></o:p></p>
<p><b>Table 2: Delivery time from kitchen to city area (in minutes)</b></p>
<table width="447" height="201" style="width: 40%;">
<tbody>
<tr>
<td width="10%" style="text-align: center; border: 1px solid black;"></td>
<td width="10%" style="text-align: center; border: 1px solid black;">Kitchen 1</td>
<td width="10%" style="text-align: center; border: 1px solid black;">Kitchen 2</td>
</tr>
<tr>
<td width="10%" style="text-align: center; border: 1px solid black;">Area 1</td>
<td width="10%" style="text-align: center; border: 1px solid black;">17</td>
<td width="10%" style="text-align: center; border: 1px solid black;">7</td>
</tr>
<tr>
<td width="10%" style="text-align: center; border: 1px solid black;">Area 2</td>
<td width="10%" style="text-align: center; border: 1px solid black;">5</td>
<td width="10%" style="text-align: center; border: 1px solid black;">8</td>
</tr>
<tr>
<td width="10%" style="text-align: center; border: 1px solid black;">Area 3</td>
<td width="10%" style="text-align: center; border: 1px solid black;">16</td>
<td width="10%" style="text-align: center; border: 1px solid black;">19</td>
</tr>
<tr>
<td width="10%" style="text-align: center; border: 1px solid black;">Area 4</td>
<td width="10%" style="text-align: center; border: 1px solid black;">16</td>
<td width="10%" style="text-align: center; border: 1px solid black;">5</td>
</tr>
<tr>
<td width="10%" style="text-align: center; border: 1px solid black;">Area 5</td>
<td width="10%" style="text-align: center; border: 1px solid black;">12</td>
<td width="10%" style="text-align: center; border: 1px solid black;">15</td>
</tr>
</tbody>
</table>
<p></p>
<p>Minimizing delivery time is of utmost importance to keep customers happy. Food-On-Wheels has hired you to help them optimize their last-mile logistics. <strong> Your first task is to create a model that helps them decide which area to serve from which kitchen to minimize total delivery time.</strong></p>
<a href="https://youtu.be/ah0zjr2YPfM">FOW P1 Video Walkthrough Link</a><br/>
<iframe width="560" height="315"
    src="https://www.youtube.com/embed/ah0zjr2YPfM" frameborder="0" allowfullscreen>
</iframe>

In [2]:
from scx.optimize import Model

In [3]:
transport = [
    {
        'origin_name':'K1',
        'destination_name':'A1',
        'time': 17,
    },
    {
        'origin_name':'K1',
        'destination_name':'A2',
        'time': 5,
    },
    {
        'origin_name':'K1',
        'destination_name':'A3',
        'time': 16,
    },
    {
        'origin_name':'K1',
        'destination_name':'A4',
        'time': 16,
    },
    {
        'origin_name':'K1',
        'destination_name':'A5',
        'time': 12,
    },
    {
        'origin_name':'K2',
        'destination_name':'A1',
        'time': 7,
    },
    {
        'origin_name':'K2',
        'destination_name':'A2',
        'time': 8,
    },
    {
        'origin_name':'K2',
        'destination_name':'A3',
        'time': 19,
    },
    {
        'origin_name':'K2',
        'destination_name':'A4',
        'time': 5,
    },
    {
        'origin_name':'K2',
        'destination_name':'A5',
        'time': 15,
    },
]

kitchens = [
    {
        'name': 'K1',
        'limit': 250
    },
    {
        'name': 'K2',
        'limit': 350
    },
]

demand = [
    {
        'name':'A1',
        'demand':101
    },
    {
        'name':'A2',
        'demand':61
    },
    {
        'name':'A3',
        'demand':101
    },
    {
        'name':'A4',
        'demand':87
    },
    {
        'name':'A5',
        'demand':50
    },
]

In [4]:
for t in transport:
    # Create decision variables for each item in transport
    t['amt']=Model.variable(name=f"{t['origin_name']}__{t['destination_name']}__amt", lowBound=0)

In [5]:
# Initialize the model
my_model = Model(name="FoW", sense='minimize')


# Add the Objective Fn
my_model.add_objective(
    fn=Model.sum([t['amt']*t['time'] for t in transport])
)

# Add Constraints
## Demand Constraint
for d in demand:
    my_model.add_constraint(
        name=f"{d['name']}__demand",
        fn=Model.sum([t['amt'] for t in transport if t['destination_name']==d['name']]) >= d['demand']
    )

## Kitchen Capacity Constraint
for k in kitchens:
    my_model.add_constraint(
        name=f"{k['name']}__kitchen_capacity",
        fn=Model.sum([t['amt'] for t in transport if t['origin_name']==k['name']]) <= k['limit']
    )

# Solve the model
my_model.solve()

In [6]:
my_model.show_formulation()

FoW:
MINIMIZE
17*K1__A1__amt + 5*K1__A2__amt + 16*K1__A3__amt + 16*K1__A4__amt + 12*K1__A5__amt + 7*K2__A1__amt + 8*K2__A2__amt + 19*K2__A3__amt + 5*K2__A4__amt + 15*K2__A5__amt + 0
SUBJECT TO
A1__demand: K1__A1__amt + K2__A1__amt >= 101

A2__demand: K1__A2__amt + K2__A2__amt >= 61

A3__demand: K1__A3__amt + K2__A3__amt >= 101

A4__demand: K1__A4__amt + K2__A4__amt >= 87

A5__demand: K1__A5__amt + K2__A5__amt >= 50

K1__kitchen_capacity: K1__A1__amt + K1__A2__amt + K1__A3__amt + K1__A4__amt
 + K1__A5__amt <= 250

K2__kitchen_capacity: K2__A1__amt + K2__A2__amt + K2__A3__amt + K2__A4__amt
 + K2__A5__amt <= 350

VARIABLES
K1__A1__amt Continuous
K1__A2__amt Continuous
K1__A3__amt Continuous
K1__A4__amt Continuous
K1__A5__amt Continuous
K2__A1__amt Continuous
K2__A2__amt Continuous
K2__A3__amt Continuous
K2__A4__amt Continuous
K2__A5__amt Continuous



In [7]:
# Show the outputs
my_model.show_outputs()

{'objective': 3663.0,
 'status': 'Optimal',
 'variables': {'K1__A1__amt': 0.0,
               'K1__A2__amt': 61.0,
               'K1__A3__amt': 101.0,
               'K1__A4__amt': 0.0,
               'K1__A5__amt': 50.0,
               'K2__A1__amt': 101.0,
               'K2__A2__amt': 0.0,
               'K2__A3__amt': 0.0,
               'K2__A4__amt': 87.0,
               'K2__A5__amt': 0.0}}
